# ALIGNED project: GSA correlation tutorial

**Aligning Life Cycle Assessment methods and bio-based sectors for improved environmental performance**

[http://www.alignedproject.eu/](http://www.alignedproject.eu/)

_Horizon Europe grant agreement N° 101059430. Views and opinions expressed are however those of the author(s) only and do not necessarily reflect those of the European Union or the European Research Executive Agency._ 


## WP1 Shared modelling framework and learnings
### Task 1.4 Framework for interpreting uncertainty

#### Deliverable 1.2 Description of scientific methods

#### Tutorial for performing Global Sensitivity Analysis using the correlation approach

#### Massimo Pizzol, Aalborg University (AAU), 2024

This notebook show how to perform a simple Global Sensitivity Analysis (GSA) for a example product system of a biobased product and using correlation analysis to quantify the defree of sensitivity.

In [1]:
# Importing packages
import brightway2 as bw
import pandas as pd
import numpy as np
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [2]:
# open a project with ecoinvent v.3.9.1 consequential system model
bw.projects.set_current('advlca23')

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [3]:
# Import the dummy product system

# import data from csv

#mydata = pd.read_csv('LCI1-bw-format.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb = mydata.copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [4]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bw.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

Writing activities to SQLite3 database:
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2024 20:45:44
  Finished: 03/14/2024 20:45:44
  Total time elapsed: 00:00:00
  CPU %: 53.60
  Memory %: 1.26


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [5]:
# check what foreground activities are included
for act in bw.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2


In [6]:
# More info 
myact = bw.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'process',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'}

In [7]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Notes': 'Input of manufacturing',
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [8]:
# calculation of static LCA score
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
myact = bw.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bw.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

121.67148851736351


### Now perform global sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input, in this case, only 5 values.
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) A correlation is estimated between model input values and output values.

#### Step 1
A sample of values for each parameter.

In [9]:
# Can be done in many ways, here very basic using lists.
par1_values = [-1.25, -1.10, -1, -0.9, -0.75] # In bomass growth, value of CO2 uptake
par2_values = [0.1, 0.3, 0.5, 0.7, 0.9] # In biomass processing, amount of energy used.
par3_values = [20,30,40,50,60]  # In use of product, amount of manufactured product needed
par4_values = [1.25, 1.10, 1, 0.9, 0.75] # In use of product, amount of manufactured product needed

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices

In [10]:
param_samples = [(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'), 
  ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par1_values), # In bomass growth, value of CO2 uptake
(('ALIGNED-biob-prod-dummy', '403a5c32-c769-46fc-8b9a-74b8eb3c79d1'),
 ('ecoinvent 3.9 conseq', 'f4dc7d2b1d70e6c0f929ec5231c085e0'), par2_values), # In biomass processing, amount of energy used.
 (('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'), par3_values), # In use of product, amount of manufactured product needed
  (('ALIGNED-biob-prod-dummy', 'c8301e73-d521-4a89-998b-30b7e7751011'),
   ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par4_values)] # In end of life, amount of CO2 released

#### (technical note)

We can do the same in a more elegant way, taking the data directly from the BW database that we just created...


In [17]:
# In biomass growth, value of CO2 uptake
par1 = list(bw.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bw.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bw.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bw.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = 5
param_samples = [(par1['output'],par1['input'], par1.random_sample(n = n_iter)),
                 (par2['output'],par2['input'], par2.random_sample(n = n_iter)),
                 (par3['output'],par3['input'], par3.random_sample(n = n_iter)),
                 (par4['output'],par4['input'], par4.random_sample(n = n_iter))]

Let's look at the samples obtained

In [18]:
param_samples

[(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'),
  ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([-0.99042564, -1.06155594, -0.99165076, -0.78432965, -0.8968502 ])),
 (('ALIGNED-biob-prod-dummy', '403a5c32-c769-46fc-8b9a-74b8eb3c79d1'),
  ('ecoinvent 3.9 conseq', 'f4dc7d2b1d70e6c0f929ec5231c085e0'),
  array([0.51692283, 0.42683044, 0.48548735, 0.42858282, 0.49825448])),
 (('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
  array([50.78193354, 53.04013265, 42.86967965, 61.53582547, 43.45260036])),
 (('ALIGNED-biob-prod-dummy', 'c8301e73-d521-4a89-998b-30b7e7751011'),
  ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([1.10062135, 1.06218001, 0.86524342, 1.13237113, 1.18551031]))]

#### Step 2

Iteration through all parameter samples. Input values are replaced and new impact scores are calculated at each iteration.

In [19]:
# testing the loop that will be used for the simulation, iterate through parameter values
    
for i in range(0,n_iter): # iterate 5 times...
    for s in param_samples: # and for each paramater...

        if s[1][0] == "biosphere3":
            col = LCA.activity_dict[s[0]] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[s[1]] # find row index of B matrix for the exchange
            print(LCA.biosphere_matrix[row,col], s[2][i]) # print the initial and final value (to be substituted)
        
        else:
            col = LCA.activity_dict[s[0]] # find column index of A matrix for the activity
            row = LCA.activity_dict[s[1]] # find row index of A matrix for the exchange
            print(LCA.technosphere_matrix[row,col], -s[2][i]) # CHANGE OF SIGN!
        
    print("* * *")


-0.9262794226378237 -0.9904256379948362
-0.4755430112324345 -0.5169228320141506
-59.09594755244373 -50.78193354229712
1.180889086683224 1.100621347367043
* * *
-0.9262794226378237 -1.0615559375697439
-0.4755430112324345 -0.4268304413455155
-59.09594755244373 -53.040132653929696
1.180889086683224 1.0621800058746753
* * *
-0.9262794226378237 -0.9916507568735136
-0.4755430112324345 -0.4854873467333781
-59.09594755244373 -42.86967964513137
1.180889086683224 0.8652434183636484
* * *
-0.9262794226378237 -0.7843296500435579
-0.4755430112324345 -0.4285828173978779
-59.09594755244373 -61.53582547178136
1.180889086683224 1.1323711261897065
* * *
-0.9262794226378237 -0.8968501984248465
-0.4755430112324345 -0.4982544824928389
-59.09594755244373 -43.45260036401821
1.180889086683224 1.1855103131205142
* * *


In [20]:
# Implementing the loop

GSA_value_results = []


for i in range(0,n_iter): 
    for s in param_samples:

        if s[1][0] == "biosphere3":
            col = LCA.activity_dict[s[0]] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[s[1]] # find row index of B matrix for the exchange
            LCA.biosphere_matrix[row,col] = s[2][i] # substitute the value
        
        else:
            col = LCA.activity_dict[s[0]] # find column index of A matrix for the activity
            row = LCA.activity_dict[s[1]] # find row index of B matrix for the exchange
            LCA.technosphere_matrix[row,col] = -s[2][i] # substitute the value
        
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    print(LCA.score)
    GSA_value_results.append(LCA.score)

128.06793032999124
127.36347475675568
105.02655828375744
151.13295040181237
123.93670820676701


#### Step 3

Now we have both the input and output values and we can look at their correlation.

To estimate the degree of correlation use a Pearson coefficient of linear relationship between two sets of values.

See here for info: 
https://en.wikipedia.org/wiki/Pearson_correlation_coefficient
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

This approach has for example been used in:

_Kim, A., Mutel, C., Froemelt, A., 2021. Robust high-dimensional screening. Environmental Modelling & Software 105270._ https://doi.org/10.1016/j.envsoft.2021.105270


In [21]:
# calcualte correlation

corr_data = pd.DataFrame([i[2] for i in param_samples], index = ['par1','par2', 'par3', 'par4']).T
corr_data['GWI'] = GSA_value_results
corr_data.corr()

,par1,par2,par3,par4,GWI
par1,1.000000,-0.176149,0.449597,0.468925,0.663309
par2,-0.176149,1.000000,-0.685917,-0.035140,-0.514438
par3,0.449597,-0.685917,1.000000,0.378074,0.906599
par4,0.468925,-0.035140,0.378074,1.000000,0.706172
GWI,0.663309,-0.514438,0.906599,0.706172,1.000000


In this specific case "par3"  is the parameter to which the results are most sensitive to.


In [22]:
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>